In [ ]:
import pandas as pd

In [ ]:
class node:

    def __init__(self):
        self.up = self
        self.down = self
        self.left = self
        self.right = self
        self.COLUMN = None
        self.ROW = None

    def hideInCol(self):
        self.up.down = self.down
        self.down.up = self.up
        self.COL.row_count -= 1
        return self

    def showInCol(self):
        self.up.down = self
        self.down.up = self
        self.COL.row_count += 1
        return self

    def hideInRow(self):
        self.left.right = self.right
        self.right.left = self.left
        return self

    def showInRow(self):
        self.left.right = self
        self.right.left = self
        return self

    def hideWholeRow(self, start = None):
        if start == None: # if we just started the loop, don't hide me from my column, but go to my neighbor
            self.right.hideWholeRow(start = self)
            return
        if start != self: # we are not first and not last => hide me and continue
            self.hideInCol()
            self.right.hideWholeRow(start = start)
            return
        return # last possibility start == self => we have looped and can finish

    def showWholeRow(self, start = None):
        if start == None: # if we just started the loop, don't hide me from my column, but go to my neighbor
            self.left.showWholeRow(start = self)
            return
        if start != self: # we are not first and not last => hide me and continue
            self.showInCol()
            self.left.showWholeRow(start = start)
            return
        return # last possibility start == self => we have looped and can finish
    
    def getNeighbor(self, direction, by=0):
        neighbor = {
            'left'  : self.left,
            'right' : self.right,
            'up'    : self.up,
            'down'  : self.down
        }[direction]
        if by > 0:
            return neighbor.getNeighbor(direction, by = by-1)
        return neighbor




In [ ]:
class columnHeader(node):

    def __init__(self, label):
        super().__init__()
        self.label = label
        self.row_count = 0
        self.COL = self

    def hideMe(self):
        self.hideInRow()
        cell = self.down
        while cell != self:
            cell.hideWholeRow()
            cell = cell.down
        return self

    def showMe(self):
        self.showInRow()
        cell = self.up
        while cell != self:
            cell.showWholeRow()
            cell = cell.up
        return self

    def appendRow(self, row):
        row.down = self
        row.up = self.up
        self.up.down = row
        self.up = row
        row.COL = self
        self.row_count += 1
        return self

    def addToRoot(self, root):
        root.appendColumn(self)
        return self

In [ ]:
class rowHeader(node):
    
    def __init__(self, label):
        super().__init__()
        self.label = label
        self.ROW = self

    def appendColumn(self, col):
        col.right = self
        col.left = self.left
        self.left.right = col
        self.left = col
        col.ROW = self
        return self

    def addToRoot(self, root):
        root.appendRow(self)
        return self


In [ ]:
class rootNode(columnHeader, rowHeader):

    def __init__(self):
        node.__init__(self)
        self.row_count = 0
        self.label = None
        self.ROW = self
        self.COL = self

    def addToRoot(self, root):
        pass

In [ ]:
class sudoku_matrix:
    
    def __init__(self):
        self.matrix_root = rootNode()

        # generate column header labels 
        constraint_type = {
            'cell' : {
                'label_template' : 'R{i}C{j}',
                'dim_1' : 'R',
                'dim_2' : 'C',
            },
            'row' : {
                'label_template' : 'R{i}#{j}',
                'dim_1' : 'R',
                'dim_2' : '#',
            },
            'col' : {
                'label_template' : 'C{i}#{j}',
                'dim_1' : 'C',
                'dim_2' : '#',
            },
            'box' : {
                'label_template' : 'B{i}#{j}',
                'dim_1' : 'B',
                'dim_2' : '#',
            },
        }

        column_headers = { 
            c_name : {
                i : {
                    j : columnHeader(
                            c_specs['label_template'].format(i = i, j = j)
                        ).addToRoot(self.matrix_root)
                    for j in range(1,10)
                }
                for i in range(1, 10)
            }
            for c_name, c_specs in constraint_type.items()
        }

        # generate row labels

        row_labels = {
            f'R{i}C{j}#{k}' : {
                'R' : i,
                'C' : j,
                'B' : (j-1) // 3 + 3 * ((i-1) // 3) + 1,
                '#' : k
            }
            for k in range(1,10)
            for j in range(1,10)
            for i in range(1,10)
        }

        self.row_headers = {}

        # construct the sparse matrix

        for row_label, attrs in row_labels.items():

            row_header = rowHeader(attrs)
            self.matrix_root.appendRow(row_header)

            for constraint_name, c_specs in constraint_type.items():

                cell = node()
                row_header.appendColumn(cell)
                d1 = c_specs['dim_1'] # can be: 'R', 'C', 'B' or '#'
                d2 = c_specs['dim_2'] # can be: 'R', 'C', 'B' or '#'
                
                column_header = column_headers[constraint_name][attrs[d1]][attrs[d2]]
                column_header.appendRow(cell)

            self.row_headers[row_label] = row_header


In [ ]:
# file I/O utilities

def read_sudoku(filename):
    with open(filename, 'r') as f:
        lines = f.readlines(121)
    tab = [
        {
            'R' : l_id + 1,
            'C' : ch_id + 1,
            '#' : ch
        }
        for l_id, line in enumerate(lines)
        for ch_id, ch in enumerate(line)
        if (ch != '-' and ch != '\n')
    ]
    return tab

def print_sudoku(sudoku, pretty=False):
    tab = [['-' for i in range(1,10)] for j in range(1,10)]
    for item in sudoku:
        tab[item['R']-1][item['C']-1] = str(item['#'])
    
    lines = [''.join(line) for line in tab]

    if pretty == True:
        lines = [f"{l[0:3]} {l[3:6]} {l[6:9]}" for l in lines]
        lines[2] = lines[2] + '\n'
        lines[5] = lines[5] + '\n'
        
    res = '\n'.join(lines)
    return res


In [ ]:
def prep_solution(solution, by='row'):
    if  by == 'row':
        return [row.ROW.label for row in solution]
    if by == 'col':
        result = []
        for row in solution:
            cols = []
            row_head = row.ROW
            c = row_head.right
            while c != row_head:
                cols.append(c.COL.label)
                c = c.right
            result.append(cols)
        return result

            

In [ ]:
def search(matrix, solution=[], log=[], k=0):

    # print(f"k : {k}")

    if type(matrix) != rootNode:
        raise TypeError("Must start with matrix root node")

    # solution found if matrix is empty
    if matrix.right == matrix:
        print("ready")
        return prep_solution(solution)

    # select column
    col = matrix.right
    tmp_col = col
    while tmp_col != matrix:
        if tmp_col.row_count < col.row_count:
            col = tmp_col
        tmp_col = tmp_col.right
    print(col.label)
    # hide column 
    col.hideMe()
    # go through its each row
    row = col.down
    print(row.ROW.label)
    while row != col:
        # add row to solution
        solution.append(row)
        log.append(f"select R{row.ROW.label['R']}C{row.ROW.label['C']}={row.ROW.label['#']}")
        # hide row's columns
        cell = row.right
        while cell != row:
            if type(cell) != rowHeader:
                cell.COL.hideMe()
            cell = cell.right
        # go deeper
        result = search(matrix, solution, log, k = k+1)
        if result != None:
            return result
        # uncover columns and remove row from solution
        solution.pop()
        log.append("back")
        cell = row.left
        while cell != row:
            if type(cell) != rowHeader:
                cell.COL.showMe()
            cell = cell.left
        # next row
        row = row.down
    # # uncover column
    col.showMe()
    return None

    

In [ ]:
m = sudoku_matrix()

solution = []
log = []
t = read_sudoku('input_5.sudoku')
start_labels = [ 
    "".join(
        [
            f"{k}{e}"
            for k, e in r.items()
        ]
    )
    for r in t
]
for l in start_labels:
    row = m.row_headers[l]
    # add row to solution and hide it's columns
    solution.append(row)
    cell = row.right
    while cell != row:
        cell.COL.hideMe()
        cell = cell.right

In [ ]:
sudoku_sulution = search(m.matrix_root, solution, log)

In [ ]:
print(print_sudoku(sudoku_sulution, True))

In [ ]:
#validate
sufoku_sol_df = pd.DataFrame(sudoku_sulution)
min_c = 999
for k in ['R','C','B']:
    for i in range (1,10):
        c = sufoku_sol_df[sufoku_sol_df[k] == i]['#'].nunique()
        if c < min_c:
            min_c = c
min_c